In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import ipyparallel as ipp
print(ipp.version_info)

(8, 6, 1)


In [2]:
nproc = 4
cluster = ipp.Cluster(engines='mpi', n=nproc, shutdown_atexit=False)
print(cluster)
client = cluster.start_and_connect_sync(activate=True)
view = client[:]
client.ids

<Cluster(cluster_id='1689594517-odnd', profile='default')>
Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.50s/engine]


[0, 1, 2, 3]

In [3]:
!ipcluster list

PROFILE          CLUSTER ID                       RUNNING ENGINES LAUNCHER
default          1689594517-odnd                  True          4 MPI


In [4]:
%autopx --block --group-outputs=engine

%autopx enabled


In [6]:
import time
import numpy as np
from mpi4py.MPI import COMM_WORLD
from qtmpy.mpi import QTMComm

comm_world = QTMComm(COMM_WORLD)

def print_msg(msg: str):
    curr_time = time.strftime("%H:%M:%S", time.localtime())
    print(f"{curr_time}: {msg}")

if comm_world.size != 4:
    raise RuntimeError(
        "This example requires 4 MPI Process. "
        f"got COMM_WORLD.size = {comm.size}"
    )


In [7]:
with comm_world as comm:
    size, rank = comm.size, comm.rank
    print_msg(f"Hello from process #{rank}/{size}")
    # Proc 1 going to sleep while the rest skip ahead
    if comm.rank == 1:
        print_msg(f"process #{rank} going to sleep for 3 seconds")
        time.sleep(3)
    print_msg(f"process #{rank} is at the end of 'with' code-block")

# When exiting, all procs in comm will be in sync.
# No need for 'comm.Barrier()' here.
# So all procs will print the message below at the exact time.
print_msg(f"process #{rank} has exited 'with' code-block")

[stdout:2] 17:19:03: Hello from process #2/4
17:19:03: process #2 is at the end of 'with' code-block
17:19:06: process #2 has exited 'with' code-block


[stdout:0] 17:19:03: Hello from process #0/4
17:19:03: process #0 is at the end of 'with' code-block
17:19:06: process #0 has exited 'with' code-block


[stdout:1] 17:19:03: Hello from process #1/4
17:19:03: process #1 going to sleep for 3 seconds
17:19:06: process #1 is at the end of 'with' code-block
17:19:06: process #1 has exited 'with' code-block


[stdout:3] 17:19:03: Hello from process #3/4
17:19:03: process #3 is at the end of 'with' code-block
17:19:06: process #3 has exited 'with' code-block


%px: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.92tasks/s]


In [8]:
color, key = rank % 2, rank // 2

with comm_world.Split(color, key) as comm:
    wrld_size, wrld_rank = comm_world.size, comm_world.rank
    grp_id = comm.id_
    grp_size, grp_rank = comm.size, comm.rank
    print(f"process #{rank}/{size} is assigned to "
          f"subgroup #{grp_id} and its rank is "
          f"{grp_rank}/{grp_size}")
    

[stdout:3] process #3/4 is assigned to subgroup #1 and its rank is 1/2


[stdout:2] process #2/4 is assigned to subgroup #0 and its rank is 1/2


[stdout:1] process #1/4 is assigned to subgroup #1 and its rank is 0/2


[stdout:0] process #0/4 is assigned to subgroup #0 and its rank is 0/2


In [1]:
grp_iproc = [0, 3]

with comm_world.Incl(grp_iproc) as comm:
    if not comm.is_null:
        wrld_size, wrld_rank = comm_world.size, comm_world.rank
        grp_size, grp_rank = comm.size, comm.rank
        print_msg(f"process #{rank}/{size} is part of the subgroup"
                  f"and its rank is {grp_rank}/{grp_size}")
        print_msg(f"process #{rank} going to sleep for 3 seconds")
        time.sleep(3)
    else:
        print_msg(f"process #{rank}/{size} is not part of the subgroup")
    print_msg(f"process #{rank} is at the end of 'with' code-block")

NameError: name 'comm_world' is not defined

In [8]:
%autopx --block --group-outputs=engine

%autopx disabled


In [22]:
cluster.stop_cluster_sync()

[1:execute]
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[15], line 1
----> 1 cluster.stop_cluster_sync()

NameError: name 'cluster' is not defined
[2:execute]
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[15], line 1
----> 1 cluster.stop_cluster_sync()

NameError: name 'cluster' is not defined


%px:  50%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                               | 2/4 [00:09<00:09,  4.90s/tasks]


Received Keyboard Interrupt. Sending signal SIGINT to engines...


In [20]:
!ipcluster list

Received Keyboard Interrupt. Sending signal SIGINT to engines...
